In [1]:
import pandas as pd
import numpy as np

In [2]:
import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
# from tigramite.independence_tests.GPDC import ParCorr, GPDC, CMIknn, CMIsymb
from tigramite.independence_tests.parcorr import ParCorr
# from tigramite.independence_tests.gpdc import GPDC
from tigramite.independence_tests.gpdc_torch import GPDCtorch
from tigramite.independence_tests.cmiknn import CMIknn
from tigramite.independence_tests.cmisymb import CMIsymb

/home/dell/anaconda3/lib/python3.10/site-packages/tigramite/data_processing.py:1329: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _get_patterns(array, array_mask, patt, patt_mask, weights, dim, step, fac, N, T):
/home/dell/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import warnings
warnings.simplefilter('ignore')

In [25]:
etth1data = pd.read_csv(
    'dataset/ETT-small/ETTh1.csv'
)
etth1data = etth1data.drop('date', axis=1)
target = etth1data.pop('OT')
etth1data.insert(0, 'OT', target)
data = etth1data

In [26]:
train_proportion = 0.6
train_size = int(data.shape[0] * train_proportion)
batch_size = 2000
tau_max = 5
tau_min = 1

arrows = np.ones(data.shape[1])*-1
train_size
data = (data - data.min()) / ( data.max() - data.min())
data

,OT,HUFL,HULL,MUFL,MULL,LUFL,LULL
0,0.691018,0.615599,0.454943,0.628980,0.467510,0.556576,0.613765
1,0.636233,0.612708,0.459449,0.626458,0.464878,0.550279,0.620783
2,0.636233,0.601143,0.436920,0.621438,0.459689,0.512595,0.586144
3,0.581468,0.599698,0.450437,0.621438,0.462320,0.515693,0.599955
4,0.519656,0.605480,0.450437,0.626458,0.467510,0.521990,0.599955
...,...,...,...,...,...,...,...
17415,0.299159,0.453765,0.558574,0.458955,0.589577,0.481107,0.655196
17416,0.301955,0.371392,0.608137,0.376064,0.599956,0.487404,0.689608
17417,0.286521,0.550572,0.576597,0.572038,0.587018,0.506298,0.655196
17418,0.276679,0.689299,0.576597,0.720262,0.587018,0.500000,0.634594


In [27]:
for i in range(int(train_size // batch_size)):
    fed_data = data.iloc[i*batch_size:(i+1)*batch_size, :]
    fed_data = np.array(fed_data).astype(float)


    T, N = fed_data.shape
    dataframe = pp.DataFrame(fed_data,
                                datatime={0:np.arange(len(fed_data))}, 
                                var_names = data.columns
                            )
    gpdc = GPDCtorch(significance='analytic')
    pcmci = PCMCI(dataframe=dataframe, 
                    cond_ind_test=gpdc,
                    verbosity=0)
    results = pcmci.run_pcmci(tau_max=tau_max, tau_min=tau_min, pc_alpha=0.1, alpha_level = 0.05)
    q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'],
                                            tau_min=tau_min, 
                                            tau_max=tau_max,
                                            fdr_method='fdr_bh')

    graph = pcmci.get_graph_from_pmatrix(
        p_matrix=q_matrix,
        alpha_level=0.01,
        tau_min=tau_min,
        tau_max=tau_max
        # ,selected_links=None
    )

    for i in range(N):
        for j in range(tau_max+1):
            if results['graph'][0][i][j] != '':
                if arrows[i] == -1 or arrows[i] > j:
                    arrows[i] = j
                    break
    arrows.astype(int)

    print(arrows)

Number of devices: 1 -- Kernel partition size: 0
[ 1.  1. -1.  1.  3.  2.  2.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 4. 1. 3. 2. 2.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 4. 1. 3. 2. 2.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 3. 1. 3. 2. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 3. 1. 3. 2. 1.]


In [28]:
etth2data = pd.read_csv(
    'dataset/ETT-small/ETTh2.csv'
)
etth2data = etth2data.drop('date', axis=1)
target = etth2data.pop('OT')
etth2data.insert(0, 'OT', target)
data = etth2data

In [29]:
train_proportion = 0.6
train_size = int(data.shape[0] * train_proportion)
batch_size = 2000
tau_max = 5
tau_min = 1

arrows = np.ones(data.shape[1])*-1
train_size
data = (data - data.min()) / ( data.max() - data.min())
data

,OT,HUFL,HULL,MUFL,MULL,LUFL,LULL
0,0.671426,0.381211,0.565340,0.308820,0.451429,0.594716,0.952870
1,0.646428,0.347826,0.522796,0.277123,0.399830,0.595065,0.950079
2,0.635716,0.351710,0.544077,0.294118,0.441579,0.521097,0.914753
3,0.589287,0.361024,0.554709,0.296727,0.453722,0.561645,0.950079
4,0.560713,0.353257,0.547107,0.295093,0.459015,0.519070,0.914753
...,...,...,...,...,...,...,...
17415,0.808333,0.363351,0.545583,0.468625,0.524285,0.127312,0.878176
17416,0.826188,0.353257,0.538000,0.457848,0.508350,0.127312,0.877246
17417,0.826188,0.367234,0.538000,0.480390,0.516700,0.088476,0.873524
17418,0.804766,0.404503,0.582068,0.530716,0.576620,0.128326,0.873524


In [30]:
for i in range(int(train_size // batch_size)):
    fed_data = data.iloc[i*batch_size:(i+1)*batch_size, :]
    fed_data = np.array(fed_data).astype(float)


    T, N = fed_data.shape
    dataframe = pp.DataFrame(fed_data,
                                datatime={0:np.arange(len(fed_data))}, 
                                var_names = data.columns
                            )
    gpdc = GPDCtorch(significance='analytic')
    pcmci = PCMCI(dataframe=dataframe, 
                    cond_ind_test=gpdc,
                    verbosity=0)
    results = pcmci.run_pcmci(tau_max=tau_max, tau_min=tau_min, pc_alpha=0.1, alpha_level = 0.05)
    q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'],
                                            tau_min=tau_min, 
                                            tau_max=tau_max,
                                            fdr_method='fdr_bh')

    graph = pcmci.get_graph_from_pmatrix(
        p_matrix=q_matrix,
        alpha_level=0.01,
        tau_min=tau_min,
        tau_max=tau_max
        # ,selected_links=None
    )

    for i in range(N):
        for j in range(tau_max+1):
            if results['graph'][0][i][j] != '':
                if arrows[i] == -1 or arrows[i] > j:
                    arrows[i] = j
                    break
    arrows.astype(int)

    print(arrows)

Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]


In [8]:
ettm1data = pd.read_csv(
    'dataset/ETT-small/ETTm1.csv'
)
ettm1data = ettm1data.drop('date', axis=1)
target = ettm1data.pop('OT')
ettm1data.insert(0, 'OT', target)
data = ettm1data

In [9]:
train_proportion = 0.6
train_size = int(data.shape[0] * train_proportion)
batch_size = 2000
tau_max = 5
tau_min = 1

arrows = np.ones(data.shape[1])*-1
train_size
data = (data - data.min()) / ( data.max() - data.min())
data

,OT,HUFL,HULL,MUFL,MULL,LUFL,LULL
0,0.691885,0.612986,0.481134,0.629100,0.468044,0.556576,0.613765
1,0.690471,0.611573,0.485320,0.626693,0.465453,0.562874,0.627575
2,0.682070,0.611573,0.476949,0.626693,0.462934,0.559777,0.606973
3,0.621844,0.611573,0.476949,0.626693,0.465453,0.559777,0.606973
4,0.637254,0.610160,0.485320,0.626693,0.465453,0.550279,0.620783
...,...,...,...,...,...,...,...
69675,0.274508,0.686432,0.577399,0.715436,0.554988,0.521990,0.641386
69676,0.274508,0.703387,0.577399,0.732218,0.547359,0.506298,0.641386
69677,0.271701,0.717515,0.564843,0.750596,0.552469,0.509395,0.634594
69678,0.271701,0.735882,0.581584,0.762586,0.557579,0.553479,0.655196


In [10]:
for i in range(int(train_size // batch_size)):
    fed_data = data.iloc[i*batch_size:(i+1)*batch_size, :]
    fed_data = np.array(fed_data).astype(float)


    T, N = fed_data.shape
    dataframe = pp.DataFrame(fed_data,
                                datatime={0:np.arange(len(fed_data))}, 
                                var_names = data.columns
                            )
    gpdc = GPDCtorch(significance='analytic')
    pcmci = PCMCI(dataframe=dataframe, 
                    cond_ind_test=gpdc,
                    verbosity=0)
    results = pcmci.run_pcmci(tau_max=tau_max, tau_min=tau_min, pc_alpha=0.1, alpha_level = 0.05)
    q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'],
                                            tau_min=tau_min, 
                                            tau_max=tau_max,
                                            fdr_method='fdr_bh')

    graph = pcmci.get_graph_from_pmatrix(
        p_matrix=q_matrix,
        alpha_level=0.01,
        tau_min=tau_min,
        tau_max=tau_max
        # ,selected_links=None
    )

    for i in range(N):
        for j in range(tau_max+1):
            if results['graph'][0][i][j] != '':
                if arrows[i] == -1 or arrows[i] > j:
                    arrows[i] = j
                    break
    arrows.astype(int)

    print(arrows)

Number of devices: 1 -- Kernel partition size: 0
[ 1.  2.  5. -1. -1.  5. -1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 4. 3. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1

In [11]:
ettm2data = pd.read_csv(
    'dataset/ETT-small/ETTm2.csv'
)
ettm2data = ettm2data.drop('date', axis=1)
target = ettm2data.pop('OT')
ettm2data.insert(0, 'OT', target)
data = ettm2data

In [12]:
train_proportion = 0.6
train_size = int(data.shape[0] * train_proportion)
batch_size = 2000
tau_max = 5
tau_min = 1

arrows = np.ones(data.shape[1])*-1
train_size
data = (data - data.min()) / ( data.max() - data.min())
data

,OT,HUFL,HULL,MUFL,MULL,LUFL,LULL
0,0.671426,0.381211,0.635664,0.315315,0.438967,0.598115,0.931236
1,0.664291,0.367234,0.617841,0.303334,0.416914,0.559950,0.929730
2,0.650004,0.360246,0.606375,0.289096,0.390443,0.598461,0.929730
3,0.646428,0.329197,0.585982,0.267406,0.421330,0.598461,0.928509
4,0.646428,0.347826,0.600003,0.283915,0.388962,0.598461,0.928509
...,...,...,...,...,...,...,...
69675,0.790478,0.395966,0.631847,0.526394,0.529405,0.097126,0.853692
69676,0.790478,0.360246,0.598726,0.476215,0.475011,0.097126,0.853692
69677,0.783327,0.367234,0.614009,0.483328,0.505898,0.135637,0.853692
69678,0.783327,0.375001,0.619103,0.500163,0.509573,0.096780,0.853692


In [13]:
for i in range(int(train_size // batch_size)):
    fed_data = data.iloc[i*batch_size:(i+1)*batch_size, :]
    fed_data = np.array(fed_data).astype(float)


    T, N = fed_data.shape
    dataframe = pp.DataFrame(fed_data,
                                datatime={0:np.arange(len(fed_data))}, 
                                var_names = data.columns
                            )
    gpdc = GPDCtorch(significance='analytic')
    pcmci = PCMCI(dataframe=dataframe, 
                    cond_ind_test=gpdc,
                    verbosity=0)
    results = pcmci.run_pcmci(tau_max=tau_max, tau_min=tau_min, pc_alpha=0.1, alpha_level = 0.05)
    q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'],
                                            tau_min=tau_min, 
                                            tau_max=tau_max,
                                            fdr_method='fdr_bh')

    graph = pcmci.get_graph_from_pmatrix(
        p_matrix=q_matrix,
        alpha_level=0.01,
        tau_min=tau_min,
        tau_max=tau_max
        # ,selected_links=None
    )

    for i in range(N):
        for j in range(tau_max+1):
            if results['graph'][0][i][j] != '':
                if arrows[i] == -1 or arrows[i] > j:
                    arrows[i] = j
                    break
    arrows.astype(int)

    print(arrows)

Number of devices: 1 -- Kernel partition size: 0
[1. 5. 4. 5. 4. 4. 3.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0


RuntimeError: NaNs encountered when trying to perform matrix-vector multiplication

In [32]:
exchangedata = pd.read_csv(
    'dataset/exchange_rate/exchange_rate.csv'
)
exchangedata = exchangedata.drop('date', axis=1)
target = exchangedata.pop('OT')
exchangedata.insert(0, 'OT', target)
data = exchangedata

In [33]:
train_proportion = 0.6
train_size = int(data.shape[0] * train_proportion)
batch_size = 2000
tau_max = 5
tau_min = 1

arrows = np.ones(data.shape[1])*-1
train_size
data = (data - data.min()) / ( data.max() - data.min())
data

,OT,0,1,2,3,4,5,6
0,0.408496,0.488023,0.445104,0.514050,0.103674,0.792386,0.084053,0.005351
1,0.410540,0.482048,0.443990,0.512794,0.102847,0.792386,0.087651,0.000447
2,0.417286,0.489961,0.465495,0.512638,0.121012,0.792386,0.103771,0.008040
3,0.416672,0.488831,0.474075,0.514835,0.123568,0.792386,0.100604,0.000000
4,0.419739,0.487054,0.491903,0.514678,0.130396,0.792386,0.098733,0.011635
...,...,...,...,...,...,...,...,...
7583,0.608498,0.379816,0.011879,0.251411,0.510450,0.267406,0.323258,0.533590
7584,0.618422,0.385218,0.013287,0.259171,0.519321,0.267919,0.336931,0.539171
7585,0.618916,0.387411,0.025156,0.267690,0.527982,0.269738,0.332182,0.542835
7586,0.612265,0.383580,0.024927,0.265464,0.523013,0.269707,0.331174,0.541290


In [34]:
for i in range(int(train_size // batch_size)):
    fed_data = data.iloc[i*batch_size:(i+1)*batch_size, :]
    fed_data = np.array(fed_data).astype(float)


    T, N = fed_data.shape
    dataframe = pp.DataFrame(fed_data,
                                datatime={0:np.arange(len(fed_data))}, 
                                var_names = data.columns
                            )
    gpdc = GPDCtorch(significance='analytic')
    pcmci = PCMCI(dataframe=dataframe, 
                    cond_ind_test=gpdc,
                    verbosity=0)
    results = pcmci.run_pcmci(tau_max=tau_max, tau_min=tau_min, pc_alpha=0.1, alpha_level = 0.05)
    q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'],
                                            tau_min=tau_min, 
                                            tau_max=tau_max,
                                            fdr_method='fdr_bh')

    graph = pcmci.get_graph_from_pmatrix(
        p_matrix=q_matrix,
        alpha_level=0.01,
        tau_min=tau_min,
        tau_max=tau_max
        # ,selected_links=None
    )

    for i in range(N):
        for j in range(tau_max+1):
            if results['graph'][0][i][j] != '':
                if arrows[i] == -1 or arrows[i] > j:
                    arrows[i] = j
                    break
    arrows.astype(int)

    print(arrows)

Number of devices: 1 -- Kernel partition size: 0
[ 1. -1. -1. -1.  3. -1.  1.  1.]
Number of devices: 1 -- Kernel partition size: 0
[ 1. -1.  3.  5.  3. -1.  1.  1.]


In [35]:
metrodata = pd.read_csv('dataset/Metro_Interstate_Traffic_Volume.csv').drop(['weather_description', 'date_time'], axis=1)
target = metrodata.pop('traffic_volume')
metrodata.insert(0, 'OT', target)
metrodata.loc[metrodata['holiday'] == 'None', 'holiday'] = 0
metrodata.loc[metrodata['holiday'] != 0, 'holiday'] = 1
metrodata['holiday'] = metrodata['holiday'].astype('int64')
metrodata['weather_main'] = metrodata['weather_main'].map({'Clouds':0. , 'Clear':1., 'Rain':2., 'Drizzle':3., 'Mist':4., 'Haze':5., 'Fog':6.,
                                                            'Thunderstorm':7., 'Snow':8., 'Squall':9., 'Smoke':10.})
data = metrodata

In [36]:
train_proportion = 0.6
train_size = int(data.shape[0] * train_proportion)
batch_size = 2000
tau_max = 5
tau_min = 1

arrows = np.ones(data.shape[1])*-1
train_size
data = (data - data.min()) / ( data.max() - data.min())
data

,OT,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main
0,0.761676,0.0,0.929726,0.0,0.0,0.40,0.0
1,0.620330,0.0,0.933209,0.0,0.0,0.75,0.0
2,0.654808,0.0,0.933918,0.0,0.0,0.90,0.0
3,0.690385,0.0,0.935692,0.0,0.0,0.90,0.0
4,0.675549,0.0,0.938949,0.0,0.0,0.75,0.0
...,...,...,...,...,...,...,...
48199,0.486676,0.0,0.914148,0.0,0.0,0.75,0.0
48200,0.382005,0.0,0.911923,0.0,0.0,0.90,0.0
48201,0.296566,0.0,0.911826,0.0,0.0,0.90,0.7
48202,0.199176,0.0,0.909762,0.0,0.0,0.90,0.0


In [37]:
for i in range(int(train_size // batch_size)):
    fed_data = data.iloc[i*batch_size:(i+1)*batch_size, :]
    fed_data = np.array(fed_data).astype(float)


    T, N = fed_data.shape
    dataframe = pp.DataFrame(fed_data,
                                datatime={0:np.arange(len(fed_data))}, 
                                var_names = data.columns
                            )
    gpdc = GPDCtorch(significance='analytic')
    pcmci = PCMCI(dataframe=dataframe, 
                    cond_ind_test=gpdc,
                    verbosity=0)
    results = pcmci.run_pcmci(tau_max=tau_max, tau_min=tau_min, pc_alpha=0.1, alpha_level = 0.05)
    q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'],
                                            tau_min=tau_min, 
                                            tau_max=tau_max,
                                            fdr_method='fdr_bh')

    graph = pcmci.get_graph_from_pmatrix(
        p_matrix=q_matrix,
        alpha_level=0.01,
        tau_min=tau_min,
        tau_max=tau_max
        # ,selected_links=None
    )

    for i in range(N):
        for j in range(tau_max+1):
            if results['graph'][0][i][j] != '':
                if arrows[i] == -1 or arrows[i] > j:
                    arrows[i] = j
                    break
    arrows.astype(int)

    print(arrows)

Number of devices: 1 -- Kernel partition size: 0
[ 1.  1.  1. -1. -1.  3.  2.]
Number of devices: 1 -- Kernel partition size: 0
[ 1.  1.  1. -1. -1.  3.  2.]
Number of devices: 1 -- Kernel partition size: 0
[ 1.  1.  1. -1. -1.  3.  2.]
Number of devices: 1 -- Kernel partition size: 0
[ 1.  1.  1.  4. -1.  3.  1.]
Number of devices: 1 -- Kernel partition size: 0
[ 1.  1.  1.  2. -1.  1.  1.]
Number of devices: 1 -- Kernel partition size: 0
[ 1.  1.  1.  2. -1.  1.  1.]
Number of devices: 1 -- Kernel partition size: 0
[ 1.  1.  1.  2. -1.  1.  1.]
Number of devices: 1 -- Kernel partition size: 0
[ 1.  1.  1.  1. -1.  1.  1.]
Number of devices: 1 -- Kernel partition size: 0
[ 1.  1.  1.  1. -1.  1.  1.]
Number of devices: 1 -- Kernel partition size: 0
[ 1.  1.  1.  1. -1.  1.  1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 1. 1. 1. 1

In [4]:
pmdata = pd.read_csv('dataset/beijing_air.csv')
pmdata['date'] = pd.to_datetime(pmdata['date'])
pmdata['year'] = pmdata['date'].apply(lambda x:x.year)
pmdata['month'] = pmdata['date'].apply(lambda x:x.month)
pmdata['day'] = pmdata['date'].apply(lambda x:x.day)
pmdata['hour'] = pmdata['date'].apply(lambda x:x.hour)
pmdata = pmdata.drop(['date', 'wnd_dir'], axis=1)
pmdata.dtypes

data = pmdata

In [5]:
train_proportion = 0.6
train_size = int(data.shape[0] * train_proportion)
batch_size = 2000
tau_max = 5
tau_min = 1

arrows_ = np.ones(data.shape[1])*-1
train_size

26280

In [6]:
data = (data - data.min()) / ( data.max() - data.min())
data

,pollution,dew,temp,press,wnd_spd,snow,rain,year,month,day,hour
0,0.129779,0.352941,0.245902,0.527273,0.002290,0.000000,0.0,0.0,0.0,0.033333,0.000000
1,0.148893,0.367647,0.245902,0.527273,0.003811,0.000000,0.0,0.0,0.0,0.033333,0.043478
2,0.159960,0.426471,0.229508,0.545455,0.005332,0.000000,0.0,0.0,0.0,0.033333,0.086957
3,0.182093,0.485294,0.229508,0.563636,0.008391,0.037037,0.0,0.0,0.0,0.033333,0.130435
4,0.138833,0.485294,0.229508,0.563636,0.009912,0.074074,0.0,0.0,0.0,0.033333,0.173913
...,...,...,...,...,...,...,...,...,...,...,...
43795,0.008048,0.250000,0.278689,0.781818,0.395659,0.000000,0.0,1.0,1.0,1.000000,0.826087
43796,0.010060,0.264706,0.262295,0.781818,0.405588,0.000000,0.0,1.0,1.0,1.000000,0.869565
43797,0.010060,0.264706,0.262295,0.781818,0.413996,0.000000,0.0,1.0,1.0,1.000000,0.913043
43798,0.008048,0.264706,0.245902,0.781818,0.420866,0.000000,0.0,1.0,1.0,1.000000,0.956522


In [7]:
for i in range(int(train_size // batch_size)):
    fed_data = data.iloc[i*batch_size:(i+1)*batch_size, :]
    fed_data = np.array(fed_data).astype(float)


    T, N = fed_data.shape
    dataframe = pp.DataFrame(fed_data,
                                datatime={0:np.arange(len(fed_data))}, 
                                var_names = data.columns
                            )
    gpdc = GPDCtorch(significance='analytic')
    pcmci = PCMCI(dataframe=dataframe, 
                    cond_ind_test=gpdc,
                    verbosity=0)
    results = pcmci.run_pcmci(tau_min=tau_min, tau_max=tau_max, pc_alpha=0.1, alpha_level = 0.05)
    q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'],
                                            tau_min=tau_min,
                                            tau_max=tau_max,
                                            fdr_method='fdr_bh')

    graph = pcmci.get_graph_from_pmatrix(
        p_matrix=q_matrix,
        alpha_level=0.01,
        tau_min=tau_min,
        tau_max=tau_max
        # ,selected_links=None
    )

    for i in range(N):
        for j in range(tau_max+1):
            if results['graph'][0][i][j] != '':
                if arrows_[i] == -1 or arrows_[i] > j:
                    arrows_[i] = j
                    break
    arrows_.astype(int)

    print(arrows_)

Number of devices: 1 -- Kernel partition size: 0
[ 1.  1. -1.  3. -1.  1.  1. -1.  3.  2.  1.]
Number of devices: 1 -- Kernel partition size: 0
[ 1.  1. -1.  3.  3.  1.  1. -1.  1.  1.  1.]
Number of devices: 1 -- Kernel partition size: 0
[ 1.  1. -1.  3.  3.  1.  1. -1.  1.  1.  1.]
Number of devices: 1 -- Kernel partition size: 0
[ 1.  1.  2.  1.  2.  1.  1. -1.  1.  1.  1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel partition size: 0
[1. 1. 2. 1. 1. 1. 1. 1. 1. 1. 1.]
Number of devices: 1 -- Kernel p